In [65]:
import pandas as pd
from PIL import Image
import numpy as np
import os
import cv2

In [66]:
df = pd.read_parquet('../datasets/cleaned/polyvore_v1.parquet')
df

,outfitName,outfitViews,wearType,type,outfitLikes,date,set_id,outfitDesc,clothingItemIndex,clothingItemName,clothingItemPrice,clothingItemLikes,clothingItemCategoryId,imagePath
7,Being a Vans shoe model with Luke. Idk about t...,188,2.0,top,9,Two years,120161271,A fashion look from April 2014 featuring destr...,1,nirvana distressed t-shirt,10.0,1290,21,raw/images/120161271/1.jpg
8,Being a Vans shoe model with Luke. Idk about t...,188,4.0,pants,9,Two years,120161271,A fashion look from April 2014 featuring destr...,2,rag bone rock w/ black skinny jeans,235.0,17406,237,raw/images/120161271/2.jpg
15,These Chanel bags is a bad habit .x,562,2.0,top,32,Two years,143656996,12.19.14,1,monki singlet,16.0,20094,104,raw/images/143656996/1.jpg
17,These Chanel bags is a bad habit .x,562,4.0,pants,32,Two years,143656996,12.19.14,3,topshop moto joni high rise skinny jeans,65.0,23324,237,raw/images/143656996/3.jpg
19,These Chanel bags is a bad habit .x,562,1.0,bag,32,Two years,143656996,12.19.14,5,pre-owned chanel shoulder bag,2450.0,4489,37,raw/images/143656996/5.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142460,Army,281,3.0,outwear,30,One year,187953065,A fashion look from January 2016 by catherine-...,2,yoins army green buttoned slit lapel coat,33.0,431,24,raw/images/187953065/2.jpg
142461,Army,281,4.0,skirt,30,One year,187953065,A fashion look from January 2016 by catherine-...,3,topshop wool blend a-line skirt,19.0,1456,9,raw/images/187953065/3.jpg
142462,Army,281,5.0,shoes,30,One year,187953065,A fashion look from January 2016 by catherine-...,4,ralph lauren collection canvas sandals,230.0,831,46,raw/images/187953065/4.jpg
142463,Army,281,1.0,bag,30,One year,187953065,A fashion look from January 2016 by catherine-...,5,alexander mcqueen hunter green black mini leat...,1010.0,16,37,raw/images/187953065/5.jpg


In [63]:
images_dir = "../datasets/raw/images"

"""
get the pixel values from biggest images in data
maximum width and maximum height
"""

max_w = 0
max_w_img_path = ""
max_h = 0
max_h_img_path = ""

for root, dirs, files in os.walk(images_dir):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)
            
            with Image.open(image_path) as image:
                image_w, image_h = image.size
                
                if image_w > max_w:
                    max_w = image_w
                    max_w_img_path = image_path
                    
                if image_h > max_h:
                    max_h = image_h
                    max_h_img_path = image_path

print("Max width:", max_w, "image_path:", max_w_img_path)
print("Max height:", max_h, "image_path:", max_h_img_path)
    

Max width: 640 image_path: ../datasets/raw/images/36405576/0.jpg
Max height: 640 image_path: ../datasets/raw/images/36405576/0.jpg


In [85]:

"""resize images to 256x256 pixels, keep the original aspect ratio of image and fill the void space with white"""
def resize_images(image_path, output_dir, size):
    # Load the image
    image = cv2.imread(image_path)
    
    # Check if image is loaded successfully
    if image is None:
        print(f"Error: Unable to load image from {image_path}")
        return
    
    # Get original dimensions
    height, width, _ = image.shape
    # Calculate aspect ratio
    aspect_ratio = width / height
    
    # Determine new dimensions based on size and aspect ratio
    if width > height:
        new_width = size
        new_height = int(size / aspect_ratio)
    else:
        new_height = size
        new_width = int(size * aspect_ratio)
    
    # Resize the image while maintaining aspect ratio
    resized_image = cv2.resize(image, (new_width, new_height))
    # Create a white canvas of size x size
    canvas = np.ones((size, size, 3), dtype=np.uint8) * 255
    # Calculate where to place the resized image on the canvas
    x_offset = (size - new_width) // 2
    y_offset = (size - new_height) // 2
    # Paste the resized image onto the canvas
    canvas[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = resized_image
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    # Get the image filename from the original path
    filename = os.path.basename(image_path)
    # Save image to output path
    cv2.imwrite(os.path.join(output_dir, filename), canvas)
    
# Define the output directory
output_dir_base = "../datasets/resized/images/"

# Define the size for resizing
size = 256

# Apply the resizing function to each row in the DataFrame
for index, row in df.iterrows():
    set_id = row['set_id']
    clothingItemIndex = row['clothingItemIndex']
    output_dir = f"{output_dir_base}/{set_id}"
    input_path = f"../datasets/{row['imagePath']}"  # Adjust the input path
    resize_images(input_path, output_dir, size)

In [86]:
"""check if all images from the dataframe are saved to the correct location in the new file system for resized images"""

output_dir_base = "../datasets/resized/images/"
missing_count = 0
missing_files = []

for index, row in df.iterrows():
    set_id = row['set_id']
    clothingItemIndex = row['clothingItemIndex']
    output_dir = f"{output_dir_base}/{set_id}"
    
    # Check if the resized image file exists
    if not os.path.exists(output_dir):
        missing_count += 1
        missing_files.append(output_dir)  # Store the path of missing file
        print(f"Resized image file is missing for {output_dir}")

# Print the total count of missing files
print(f"Total missing files: {missing_count}")

Total missing files: 0
